In [5]:
from __future__ import annotations
import os
import uuid
import chromadb
# Import สิ่งที่จำเป็นจาก ChromaDB
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction
from chromadb.utils.data_loaders import ImageLoader

In [6]:
# ---------------- Config ----------------
PERSIST_DIR = "./chroma_multimodal_local_dataloader" # เปลี่ยน path เพื่อไม่ให้ปนกับของเก่า
COLLECTION_NAME = "pets_local_dataloader"

DOG_IMAGES = ["images/dog1.png", "images/dog2.png", "images/dog3.png"]
CAT_IMAGES = ["images/cat1.png", "images/cat2.png", "images/cat3.png"]

In [7]:
# Step 1 - เตรียม path ของรูปและ metadata
all_image_paths = DOG_IMAGES + CAT_IMAGES
labels = ["dog"] * len(DOG_IMAGES) + ["cat"] * len(CAT_IMAGES)
doc_ids = [str(uuid.uuid4()) for _ in all_image_paths]

# --- ส่วนที่แก้ไข ---
# สร้าง document description
document_descriptions = [f"A photo of a {lbl}" for lbl in labels]
# เอามารวมใน metadatas แทน
metadatas = [
    {
        "label": lbl,
        "filename": os.path.basename(path),
        "description": desc # เพิ่ม description เข้าไปใน metadata
    }
    for lbl, path, desc in zip(labels, all_image_paths, document_descriptions)
]

In [8]:
# Step 2 - Setup ChromaDB Client
# ใช้ PersistentClient เพื่อให้บันทึกข้อมูลลงดิสก์
client = chromadb.PersistentClient(path=PERSIST_DIR)

# Step 3 - สร้าง/เรียกใช้ Collection โดยระบุ Embedding Function และ Data Loader
# ChromaDB จะใช้ OpenCLIP model ซึ่งเทียบเท่ากับ "clip-ViT-B-32" โดยอัตโนมัติ
collection = client.get_or_create_collection(
    name=COLLECTION_NAME,
    embedding_function=OpenCLIPEmbeddingFunction(), # บอกให้ใช้ OpenCLIP
    data_loader=ImageLoader(),                     # บอกให้ใช้ ImageLoader
    metadata={"hnsw:space": "cosine"}
)

c:\Users\user\miniconda3\envs\vector-db\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\miniconda3\envs\vector-db\Lib\site-packages\huggingface_hub\file_download.py:121: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--laion--CLIP-ViT-B-32-laion2B-s34B-b79K. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In

In [ ]:
# Step 4 - เพิ่มรูปเข้า DB โดยใช้ URIs
# ไม่ต้องแปลงรูปเป็น embedding เองแล้ว! แค่ส่ง path ของไฟล์ไป
collection.add(
    ids=doc_ids,
    uris=all_image_paths, # <--- จุดสำคัญคือตรงนี้
    metadatas=metadatas
)
print(f"Indexed {len(doc_ids)} images into collection '{collection.name}'.")

In [ ]:
# Step 5 - ทดสอบ Query ด้วยข้อความ
# ไม่ต้องแปลง query เป็น embedding เองเช่นกัน
query_text = "dog"
res = collection.query(
  query_texts=[query_text], # <--- ส่งข้อความ query เข้าไปตรงๆ
  n_results=4,
  include=["metadatas", "documents", "distances"]
)

# แสดงผลลัพธ์
print(f"\n=== Query: '{query_text}' ===")
for rank, (meta, doc, dist) in enumerate(zip(res["metadatas"][0], res["documents"][0], res["distances"][0]), start=1):
  print(f"#{rank} -> {meta['filename']} ({meta['label']}) distance={dist:.4f}")
  print("desc:", doc)